# extract audio feature

## 01 test sample

In [13]:
import os
import csv

In [17]:
path = '/home/dato/workspace/emotional_speech_recognition/data/raw/MSP_IMPROV/Audio/session1/S01A/P/'
file_name = path + 'MSP-IMPROV-S01A-F01-P-FM01.wav'

In [18]:
out_file = '../data/processed/MSP_IMPROV/test.csv'

In [19]:
# cmd = 'SMILExtract -C /home/dato/program/opensmile-2.3.0/config/modified_prosodyShsViterbiLoudness.conf -I ' + file_name + ' -O ' + out_file + ' -headercsv 1'

#cmd = 'SMILExtract -C /home/dato/program/opensmile-2.3.0/config/modified_MFCC12_E_D_A_Z.conf -I ' + file_name + ' -O ' + out_file  + ' -headercsv 1'
# cmd = 'SMILExtract -C /home/dato/program/opensmile-2.3.0/config/modified_MFCC12_E_D_A.conf -I ' + file_name + ' -csvoutput ' + out_file  + ' -headercsv 0' # MFCC12EDA case

cmd = 'SMILExtract -C /home/dato/program/opensmile-2.3.0/config/modified_emobase2010.conf -I ' + file_name + ' -csvoutput ' + out_file  + ' -headercsv 1'

In [22]:
print cmd
os.system(cmd)

SMILExtract -C /home/dato/program/opensmile-2.3.0/config/modified_emobase2010.conf -I /home/dato/workspace/emotional_speech_recognition/data/raw/MSP_IMPROV/Audio/session1/S01A/P/MSP-IMPROV-S01A-F01-P-FM01.wav -csvoutput ../data/processed/MSP_IMPROV/test.csv -headercsv 1


0

In [23]:
# lines = []
# with open('../data/processed/MSP_IMPROV/processeed_prosody.csv') as f:
#     csv_reader = csv.reader(f)
#     lines = [x for x in csv_reader]        

## 01 extract MFCC / prosody / emobase feature from multiple files

In [1]:
import os
import sys
import csv
from file_util import *

### select opensmile configuration

In [2]:
# prosody
OPENSMILE_CONFIG_PATH = '~/program/opensmile-2.3.0/config/modified_prosodyShsViterbiLoudness.conf'
out_file = '../data/processed/MSP_IMPROV/prosody.csv'

    
# MFCC E D A    
# OPENSMILE_CONFIG_PATH = '/home/dato/program/opensmile-2.3.0/config/modified_MFCC12_E_D_A.conf'    
# out_file = '../data/processed/MSP_IMPROV/MFCC12EDA.csv'


# MFCC E D A Z
# OPENSMILE_CONFIG_PATH = '/home/dato/program/opensmile-2.3.0/config/modified_MFCC12_E_D_A_Z.conf'   # normalized 
# out_file = '../data/processed/MSP_IMPROV/MFCC12EDAZ.csv'
    
# emobase2010
# OPENSMILE_CONFIG_PATH = '/home/dato/program/opensmile-2.3.0/config/modified_emobase2010.conf'    
# out_file = '../data/processed/MSP_IMPROV/emobase2010.csv'
    
    
out_file    

'../data/processed/MSP_IMPROV/prosody.csv'

In [3]:
def extract_feature( list_in_file, out_file ) :
    print out_file
    
    cnt = 0    
    for in_file in list_in_file:        
        cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -O ' + out_file + ' -headercsv 0'  #MFCC12EDAZ, prosody
#         cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -csvoutput ' + out_file + ' -headercsv 0'   # MFCC12EDA
#         cmd = 'SMILExtract -C ' + OPENSMILE_CONFIG_PATH + ' -I ' + in_file + ' -csvoutput ' + out_file + ' -headercsv 0'   # emobase2010
        os.system(cmd)
        
        cnt += 1
        if cnt % 1000 == 0:
            print str(cnt) + " / " + str( len(list_in_file) )
            sys.stdout.flush()

In [4]:
# [schema] ID, transcriptions [csv]

list_files = []

path = '../data/raw/MSP_IMPROV/Audio/'
file_search(path, list_files)

sorted_list = [ [x.split('/')[-1][:-4], x] for x in list_files ]
sorted_list = sorted(sorted_list)
list_files_id = [ x[0] for x in sorted_list]
list_files_file_name = [x[1] for x in sorted_list]

extract_feature( list_files_file_name, out_file )

../data/processed/MSP_IMPROV/prosody.csv
1000 / 8438
2000 / 8438
3000 / 8438
4000 / 8438
5000 / 8438
6000 / 8438
7000 / 8438
8000 / 8438


#### extract id from audio_data

In [5]:
with open('../data/processed/MSP_IMPROV/id_from_audio.txt', 'w') as f:
    for x in list_files_id:
        f.write( x + '\n' )

## 02 verify

In [6]:
import csv
from file_util import *

In [7]:
in_file = '../data/processed/MSP_IMPROV/id_from_trans.txt'
label_trans = []
with open( in_file, 'r') as f:
    label_trans = f.readlines()
label_trans = [x.strip() for x in label_trans]    

In [8]:
in_file = '../data/processed/MSP_IMPROV/id_from_audio.txt'
label_audio = []
with open( in_file, 'r') as f:
    label_audio = f.readlines()    
label_audio = [x.strip() for x in label_audio]

In [9]:
cnt = 0
for l, t in zip(label_trans, label_audio):
    if l != t:
        cnt += 1
if cnt>0 :
    print "ERROR: " + str(cnt)
else :
    print "SUCCESS validation"

SUCCESS validation


## 03 A MFCC post-process
- each row has MFCC feature information (41 dimesion) * audio length --> can be different according to its length
- [0]frameIndex; [1]frameTime; were removed from processed feature
- (8438, 750, 39)

In [110]:
import csv
import cPickle
import numpy as np

#### 03-A load MFCC feature

In [111]:
lines = []
# with open('../data/processed/MSP_IMPROV/MFCC12EDAZ.csv') as f:
with open('../data/processed/MSP_IMPROV/MFCC12EDA.csv') as f:
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]

In [112]:
print len(lines)
print len(lines[0][0])
lines[0]

3433689
541


["'unknown';0.000000;-3.609930e+00;9.026229e+00;2.547681e-01;-5.463406e+00;-1.834702e+00;2.698720e+00;1.159871e+01;1.418919e+01;1.050633e+01;3.580090e+00;6.984715e+00;4.497300e+00;1.421603e+01;-6.127123e-01;-1.319952e+00;-1.298531e+00;-4.588038e-01;-2.369876e+00;8.665654e-01;3.195097e+00;1.638633e+00;2.841136e+00;3.032382e+00;2.460721e+00;1.832849e+00;-3.191834e-02;2.138871e-01;1.843904e-01;-2.296742e-02;2.453978e-01;2.646415e-01;-1.864607e-01;-1.344776e+00;-1.925809e-01;3.760855e-02;-2.834120e-02;-1.186853e-01;-4.962853e-01;1.288971e-02"]

#### 03-B MFCC feature -> float value

In [113]:
lines = [x[0] for x in lines]

In [114]:
lines = [x.split(';') for x in lines]

In [115]:
float_lines = [ [float(i) for i in x[1:]] for x in lines ]   # do not care the first element ('unknown')

In [116]:
# mark the index of each chunk 

chunk_index = []
for i, line in enumerate(float_lines):
    if line[0] == 0:
        chunk_index.append(i)

In [117]:
no_index_float_linex = [ x[2:] for x in lines ]   # remove first two element ('unknown' and sequence index)

In [118]:
# merge sequence 

list_MFCC = []

for i in xrange( len(chunk_index) ):
    
    if i == len(chunk_index) -1:        # last case
        list_MFCC.append( no_index_float_linex[ chunk_index[i]: ] )
    else :
        list_MFCC.append( no_index_float_linex[ chunk_index[i]:chunk_index[i+1] ] )

In [119]:
len(list_MFCC)

8438

#### 03-C numpy save  7.5s cut

In [120]:
stat = [ len(x) for x in list_MFCC ]
print np.mean(stat)
print np.std(stat)
print np.max(stat)
print np.min(stat)
print len( list_MFCC )

406.931618867
288.912489087
3189
39
8438


In [121]:
# np.save('../data/processed/IEMOCAP/processed_MFCC12EDAZ_sequenceN.npy', np.asarray(stat))
np.save('../data/processed/MSP_IMPROV/processed_MFCC12EDA_sequenceN.npy', np.asarray(stat))

In [122]:
# with open('../data/processed/IEMOCAP/processed_MFCC12EDAZ_sequenceN.txt', 'w') as f:
with open('../data/processed/MSP_IMPROV/processed_MFCC12EDA_sequenceN.txt', 'w') as f:
    for data in stat:
        f.write( str(data) + '\n' )

In [123]:
CUT = 1000

np_MFCC = np.zeros( [len( list_MFCC ), CUT, 39], dtype=np.float)
np.shape(np_MFCC)

(8438, 1000, 39)

In [124]:
for i in xrange( len(list_MFCC) ):
    
    if len( list_MFCC[i] ) > CUT:
        np_MFCC[i][:] = list_MFCC[i][:CUT]
    else:
        np_MFCC[i][:len(list_MFCC[i])] = list_MFCC[i][:]

In [125]:
# np.save('../data/processed/IEMOCAP/processed_MFCC12EDAZ.npy', np_MFCC)
np.save('../data/processed/MSP_IMPROV/processed_MFCC12EDA.npy', np_MFCC)

In [128]:
gogotmp = np.load('../data/processed/MSP_IMPROV/processed_MFCC12EDA.npy')

## 04 prosody post-process
- [0]@name string; [36]class numeric; were removed from original feature (total 37)
- (8438, 35) 

#### 04-A load prosody feature  &  conver to np format

In [20]:
import numpy as np

lines = []
with open('../data/processed/MSP_IMPROV/prosody.csv') as f:
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]
print len(lines)
print len(lines[0])
print lines[0]

8438
37
["'unknown'", '5.609369e+01', '1.562134e+02', '-5.579678e-01', '4.571930e-01', '8.057990e+01', '2.267393e+02', '1.461594e+02', '7.055344e+00', '2.864598e+01', '-7.188702e-02', '4.687895e-01', '1.841961e+01', '3.651830e+01', '1.809869e+01', '6.058857e-02', '7.685646e-01', '-5.243734e-04', '4.878892e-01', '6.884887e-01', '8.526720e-01', '1.641833e-01', '5.908735e+00', '-2.236514e+00', '-8.132513e-03', '5.731317e-01', '-9.352282e+00', '7.237075e+00', '1.658936e+01', '8.858352e-02', '1.377914e-01', '-9.319513e-05', '4.759388e-01', '7.157311e-02', '2.628319e-01', '1.912588e-01', '?']


In [21]:
np_prosody = np.zeros( [len(lines), 35], dtype=np.float)
np.shape(np_prosody)

(8438, 35)

In [22]:
for i in xrange( len(np_prosody) ):
    np_prosody[i] = lines[i][1:-1]    

In [23]:
len(lines)

8438

#### 04-B  numpy save

In [24]:
np.save('../data/processed/MSP_IMPROV/processed_prosody.npy', np_prosody)
# aaa = np.load('../data/processed/MSP_IMPROV/processed_prosody.npy')

## 05 emobase2010 post-process
- [0]frameTime aws removed from original feature (total 1583)
- (8438, 1582) 

#### 05-A load emobase2010 feature  &  conver to np format

In [26]:
lines = []
with open('../data/processed/MSP_IMPROV/emobase2010.csv') as f:
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]

lines = [x[0].split(';') for x in lines]
lines = [ [float(t) for t in x] for x in lines]

In [28]:
np_emobase = np.zeros( [len(lines), 1582], dtype=np.float)
np.shape(np_emobase)

(10039, 1582)

In [30]:
for i in xrange( len(np_emobase) ):
    np_emobase[i] = lines[i][1:]    

#### 04-B  numpy save

In [34]:
np.save('../data/processed/MSP_IMPROV/processed_emobase2010.npy', np_emobase)

In [5]:
tmp = np.load( '../data/processed/MSP_IMPROV/processed_emobase2010.npy')
np.shape(tmp)

(10039, 1582)